In [ ]:
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField # BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError # Email
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    bio = db.Column(db.Text) #important! profile, table, form
    admin = db.Column(db.Boolean) #important!
    notes = db.relationship('Note', backref='writer', lazy='dynamic') #important!

class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.utcnow, nullable=False)
    # date_created = db.Column(db.DateTime, default=datetime.utcnow)

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")


class NewNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    # note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Note Body"})
    note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("Add Note")

class UpdateNoteForm(FlaskForm):
    title = StringField("Title", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "Title"})
    # note_body = TextAreaField("Note Body", validators=[InputRequired(), Length(max=100)], render_kw={"placeholder":  "Note Body"})
    note_body = StringField(u'Text', widget=TextArea())
    submit = SubmitField("Update Note")


@app.route('/home')
@app.route('/')
def home():
    return render_template('index.html', title='Home')


@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("view_notes"))

        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password,bio='',admin=0)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))


@app.route('/new-note', methods=['GET','POST'])
@login_required
def new_note():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_note = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user, author=current_user.username)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('view_notes'))
    return render_template('new_note.html', title='New Note', form=form)

@app.route('/my-notes', methods=['GET','POST'])
@login_required
def view_notes():
    if current_user.admin:
        notes = Note.query.all()  #if admin, can see all. if user, only see their own notes
    else:
        notes = Note.query.filter_by(writer=current_user).all()
    return render_template('my_notes.html', notes=notes, title='My Notes')

@app.route('/delete-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('view_notes'))

@app.route('/update-note/<int:note_id>', methods=['GET',"POST"]) #get note id and then send it back, link to <td><a href="{{url_for('update_note', note_id=note.id)}}" class="btn btn-primary pull-right">Update</a></td>
@login_required
def update_note(note_id):
    form = UpdateNoteForm()
    note = Note.query.get_or_404(note_id)
    if request.method == 'GET':
        form.title.data = note.title
        form.note_body.data = note.note_body
    if form.validate_on_submit():  # request.method="POST"
            note.query.filter_by(id=note_id).first()
            if note:
                note.title = form.title.data                #post data returned from the form to update the table(db.session.commit)
                note.note_body = form.note_body.data
                db.session.commit()
                print("Note id=",note_id)
                print("Updated.")
                return redirect(url_for("view_notes"))

    return render_template('update_note.html', form=form, title='Update Note', id=note_id) #form=form, send it back 

if __name__ == '__main__':
    app.run('localhost', 9098)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


D:\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9098/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2021 15:51:46] "GET / HTTP/1.1" 200 -


<meta charset="utf-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>Clean Blog - Start Bootstrap Theme</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@4.6.0/dist/css/bootstrap.min.css" rel="stylesheet">
        <!-- Font Awesome icons (free version)-->

css:
     body {
      padding-top: 3.5rem;
 }

navigation bar:
 {% if current_user.is_authenticated %}
 show my note
  {% else %}
  only login and register

<!-- Bootstrap core JS-->
<script src="https://cdn.jsdelivr.net/npm/jquery@3.5.1/dist/jquery.slim.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@4.6.0/dist/js/bootstrap.bundle.min.js"></script> 
<!-- Core theme JS-->
<script src="{{ url_for('static', filename='js/scripts.js') }}"></script>
<script src="//cdn.ckeditor.com/4.16.0/standard/ckeditor.js"></script>
<script type="text/javascript">

<form method='POST' action=''>
            {{form.hidden_tag()}}
            {{form.title.label(class="form-label")}}<br>  
            {{form.title(class="form-control")}}
            {{form.note_body.label(class="form-label")}}
            {{form.note_body(rows='3',cols='100',class="form-control",id="editor")}}
            {{form.submit(class="btn btn-primary")}}
        </form>